In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

2025-04-27 11:11:33.244115: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 11:11:33.496335: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-27 11:11:33.784347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745732494.040653    4127 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745732494.112053    4127 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745732494.646554    4127 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
## Load the Ann model , scaler, onehot encoder 

model = load_model('model.h5')

## load the encoders and scalers 

with open('label_encoder_gender.pkl','rb') as file_object:
    gender_encoder = pickle.load(file_object)

with open('onehot_geography.pkl','rb') as file_object:
    geo_encoder = pickle.load(file_object)

with open('scaler.pkl','rb') as file_object:
    scaler = pickle.load(file_object)

2025-04-27 11:11:58.955316: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
# example the input data

input_data = {
    'CreditScore': 600,
    'Geography': 'France', 
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [4]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [15]:
# one hot encode 'geography'

geo_encoded = geo_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=geo_encoder.get_feature_names_out(['Geography']))
geo_encoded_df


/home/fadhil/Desktop/ML/DeepLearning/customer-churn/cenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [ ]:
# label encode 'gender'
input_data_df['Gender'] = gender_encoder.transform(input_data_df['Gender'])
input_data_df

/home/fadhil/Desktop/ML/DeepLearning/customer-churn/cenv/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ValueError: y contains previously unseen labels: 1

In [7]:
# combine onehot encoded columns with input dataFrame 
input_data_df = pd.concat([input_data_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
## scaling the input data

input_scaled = scaler.transform(input_data_df)
input_scaled

array([[-0.55012981,  0.89091075,  0.12136034, -0.68041201, -0.28051905,
         0.84584804,  0.62670381,  0.968496  , -0.91572441,  1.00400803,
        -0.57427105, -0.58350885]])

In [14]:
## prediction 

prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step


array([[0.02693672]], dtype=float32)

In [15]:
## prediction probability 

prediction_proba = prediction[0][0]

In [16]:
prediction_proba

np.float32(0.026936715)

In [17]:
if prediction_proba > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
